# Redes Neuronales Artificales


### Instalar Theano
pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

### Instalar Tensorflow y Keras
conda install -c conda-forge keras

## Parte 1 - Pre procesado de datos

### Cómo importar las librerías


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


### Importar el data set


In [ ]:
dataset = pd.read_csv('https://raw.githubusercontent.com/joanby/deeplearning-az/master/datasets/Part%201%20-%20Artificial%20Neural%20Networks%20(ANN)/Churn_Modelling.csv')

X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [ ]:
dataset.iloc[0,:].values

array([1, 15634602, 'Hargrave', 619, 'France', 'Female', 42, 2, 0.0, 1, 1,
       1, 101348.88, 1], dtype=object)

In [ ]:
X[0,:]

array([619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)


### Codificar datos categóricos


In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(
    transformers=[
        ("Churn_Modelling",        # Un nombre de la transformación
         OneHotEncoder(categories='auto'), # La clase a la que transformar
         [1]            # Las columnas a transformar.
         )
    ], remainder='passthrough'
)

X = transformer.fit_transform(X)
X = X[:, 1:]


### Dividir el data set en conjunto de entrenamiento y conjunto de testing


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Escalado de variables


In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

## Parte 2 - Construir la RNA




In [ ]:
# Importar Keras y librerías adicionales
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Inicializar la RNA
classifier = Sequential()

# Añadir las capas de entrada y primera capa oculta
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  
                     activation = "relu", input_dim = 11))

# Añadir la segunda capa oculta
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

# Añadir la capa de salida
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

# Compilar la RNA
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

# Ajustamos la RNA al Conjunto de Entrenamiento
classifier.fit(X_train, y_train,  batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 2s 1ms/step - loss: 0.4948 - accuracy: 0.7954
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4252 - accuracy: 0.7968
Epoch 3/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4178 - accuracy: 0.8202
Epoch 4/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4122 - accuracy: 0.8305
Epoch 5/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4076 - accuracy: 0.8320
Epoch 6/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4052 - accuracy: 0.8329
Epoch 7/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4036 - accuracy: 0.8344
Epoch 8/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4020 - accuracy: 0.8338
Epoch 9/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4005 - accuracy: 0.8341
Epoch 10/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3999 - accura

## Parte 3 - Evaluar el modelo y calcular predicciones finales


### Predicción de los resultados con el Conjunto de Testing

In [ ]:
y_pred  = classifier.predict(X_test)
y_pred = (y_pred>0.5)

### Predecir una nueva observación

Utiliza nuestro modelo de RNA para predecir si el cliente con la siguiente información abandonará el banco:




*   Geografia: Francia
*   Puntaje de crédito: 600
*   Género masculino
*   Edad: 40 años de edad
*   Tenencia: 3 años.
*   Saldo: $ 60000

*   Número de productos: 2
*   ¿Este cliente tiene una tarjeta de crédito? Sí
*   ¿Es este cliente un miembro activo? Sí
*   Salario estimado: $ 50000

Entonces, ¿deberíamos decir adiós a ese cliente?

In [ ]:
new_prediction = classifier.predict(sc_X.transform(np.array([[0,0,600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
print(new_prediction)
print(new_prediction > 0.5)

[[0.07617277]]
[[False]]


### Elaborar una matriz de confusión



In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1531,   64],
       [ 236,  169]])

In [ ]:
(cm[0][0]+cm[1][1])/cm.sum()

0.85

# Parte 4 Evaluar, Mejorar y Ajustar la RNA

### Evaluar la RNA

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.layers import Dropout

In [ ]:
def build_classifier():
  # Inicializar la RNA
  classifier = Sequential()

  # Añadir las capas de entrada y primera capa oculta
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  
                       activation = "relu", input_dim = 11))

  # Añadir la segunda capa oculta
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

  # Añadir la capa de salida
  classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

  # Compilar la RNA
  classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

  # Devolver el clasificador
  return classifier

In [49]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1, verbose = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    5.0s finished
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_

In [50]:
accuracies

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [51]:
mean = accuracies.mean()
variance = accuracies.std()
print(mean)
print(variance)

nan
nan


### Mejorar la RNA

Regularizacion de Dropout para evitar el overfitting

### Ajustar la RNA

In [52]:
from sklearn.model_selection import GridSearchCV 

In [53]:
def build_classifier(optimizer):
  # Inicializar la RNA
  classifier = Sequential()

  # Añadir las capas de entrada y primera capa oculta
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  
                       activation = "relu", input_dim = 11))

  # Añadir la segunda capa oculta
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

  # Añadir la capa de salida
  classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

  # Compilar la RNA
  classifier.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])

  # Devolver el clasificador
  return classifier

In [54]:
classifier = KerasClassifier(build_fn = build_classifier)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


In [57]:
parameters = {
    'batch_size' : [25, 32],
    'epochs' : [100],
    'optimizer' : ['adam', 'rmsprop']
}

In [58]:
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)

grid_search  = grid_search.fit(X_train, y_train)
best_params  = grid_search.best_params_
best_accuracy = grid_search.best_score_

Se truncaron las últimas líneas 5000 del resultado de transmisión.
288/288 [==============================] - 1s 2ms/step - loss: 0.5862 - accuracy: 0.7954
Epoch 2/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4447 - accuracy: 0.7969
Epoch 3/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4329 - accuracy: 0.7969
Epoch 4/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4297 - accuracy: 0.7969
Epoch 5/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4281 - accuracy: 0.7969
Epoch 6/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4264 - accuracy: 0.7969
Epoch 7/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4243 - accuracy: 0.7969
Epoch 8/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4219 - accuracy: 0.7978
Epoch 9/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4199 - accuracy: 0.8182
Epoch 10/100
288/288 [==============

In [59]:
print(best_params  )
print(best_accuracy)

{'batch_size': 32, 'epochs': 100, 'optimizer': 'adam'}
0.842625
